# Feature Engineering

Preparo el dataset para ser posteriormente usado en el modelo

Indicadores a utilizar:

* Percentage price oscilator (PPO)
* Moving average convergence/divergence (MACD)
* Normalized average true range (NATR)
* Relative Strenght index (RSI)
* Bollinger Bands

## Unimos los tickers

In [2]:
# !pip install ta-lib

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
from talib import MACD, PPO, MACD, NATR, RSI, BBANDS, SMA, ATR
from tqdm import tqdm

tqdm.pandas()

/home/thomas/anaconda3/envs/dhdsblend/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Indexamos por ticker para facilitar calculos

In [20]:
dataset = pd.read_pickle("data/all_tickers_raw.pkl")
dataset.sort_index(inplace = True)
dataset.set_index('Ticker', append = True, inplace = True)
dataset.head()

Open      High       Low     Close     Adj Close  \
Date       Ticker                                                         
1962-01-02 GE      0.751202  0.763722  0.743690  0.748698  1.775989e-03   
           MO      0.000000  0.191840  0.189236  0.189236  2.221978e-04   
           CVX     0.000000  3.296131  3.244048  3.296131  5.051204e-02   
           JNJ     0.000000  0.223380  0.222222  0.223380  6.196926e-07   
           CAT     1.604167  1.619792  1.588542  1.604167  1.347026e-01   

                      Volume Currency  
Date       Ticker                      
1962-01-02 GE      2156500.0      USD  
           MO       345600.0      USD  
           CVX      105600.0      USD  
           JNJ           0.0      USD  
           CAT      163200.0      USD

## Calculamos indicadores

### Compute Moving Average Convergence/Divergence (MACD)

In [ ]:
def compute_macd(close):
    macd = MACD(close)[0]
    return macd

dataset["MACD"] = dataset.groupby('Ticker', level = 2).Close.apply(compute_macd)

In [ ]:
alua_mask = dataset.index.get_level_values('Ticker') == "ALUA.BA"

In [ ]:
sns.distplot(data = dataset[alua_mask], x = "MACD")

### Relative Strength Index (RSI)

In [ ]:
dataset["RSI"] = dataset.groupby('Ticker').Close.apply(RSI)

In [ ]:
sns.displot(data = dataset[alua_mask], x = "RSI")

### Bollinger Bands

In [ ]:
def compute_bb(close):
    high, mid, low = BBANDS(np.log1p(close), timeperiod=20)
    return pd.DataFrame({'BB_High': high, 'BB_Mid': mid, 'BB_Low': low}, index=close.index)

In [ ]:
bollinger_bands = dataset.groupby('Ticker').Close.apply(compute_bb)
dataset = dataset.join(bollinger_bands)

### Average True Range

In [ ]:
def compute_atr(stock_data):
    return ATR(stock_data.High, 
              stock_data.Low, 
              stock_data.Close, 
              timeperiod=14)

In [ ]:
dataset["ATR"] = dataset.groupby("Ticker", group_keys=False).apply(compute_atr)

### Normalized average true range

In [ ]:
dataset["NATR"] = dataset.groupby("Ticker", group_keys=False).apply(lambda x: NATR(high=x.High, low=x.Low, close=x.Close))

### Volumen en moneda

In [ ]:
dataset["Currency_Volume"] = dataset.loc[:, 'Close'].mul(dataset.loc[:, 'Volume'], axis=0)

### Retorno mensual, bimensual y trimestral

In [ ]:
def remove_outliers(x):
    return x.clip(lower=x.quantile(0.01), upper=x.quantile(0.99))

In [ ]:
dataset["Return_1w"] = dataset.sort_index().groupby("Ticker").Close.pct_change(7).pipe(remove_outliers)
dataset["Return_1m"] = dataset.sort_index().groupby("Ticker").Close.pct_change(30).pipe(remove_outliers)
dataset["Return_2m"] = dataset.sort_index().groupby("Ticker").Close.pct_change(60).pipe(remove_outliers)
dataset["Return_3m"] = dataset.sort_index().groupby("Ticker").Close.pct_change(90).pipe(remove_outliers)

In [ ]:
sns.displot(data = dataset[alua_mask], x = "Return_1m")

In [ ]:
sns.displot(data = dataset[alua_mask], x = "Return_2m")

In [ ]:
sns.displot(data = dataset[alua_mask], x = "Return_3m")

In [ ]:
sns.displot(data = dataset[alua_mask], x = "Return_1w")

### Confección de target con features

Utilizando el % de diferencia a 1, 2 y 3 meses confeccionamos el target, moviendo el periodo de holding hacia el pasado.

In [ ]:
dataset["Forward_Return_1w"] = dataset.groupby('Ticker')[f'Return_1w'].shift(-7)
dataset["Forward_Return_1m"] = dataset.groupby('Ticker')[f'Return_1m'].shift(-30)
dataset["Forward_Return_2m"] = dataset.groupby('Ticker')[f'Return_2m'].shift(-60)
dataset["Forward_Return_3m"] = dataset.groupby('Ticker')[f'Return_3m'].shift(-90)

Creamos feature de fechas

In [ ]:
dataset['Year'] = dataset.apply(lambda frame: frame.index.get_level_values(0).year).Close
dataset['Month'] = dataset.apply(lambda frame: frame.index.get_level_values(0).month).Close
dataset['Weekday'] = dataset.apply(lambda frame: frame.index.get_level_values(0).weekday).Close

Eliminamos los indices de ticker para llevarlos a sus respectivas columnas

In [ ]:
dataset.reset_index(inplace = True, level = 1)
dataset.sample(5)

Ordenamos por fecha para evitar esta reingenieria durante los procesos de entrenamiento e iteracion de modelos

In [ ]:
dataset.sort_index(inplace = True)
dataset.index.is_monotonic

### Salvado de los datos

In [ ]:
dataset.shape

In [ ]:
dataset.info(null_counts = True)

In [17]:
dataset.shape

(1964450, 9)

In [18]:
dataset.to_pickle("data/all_tickers_features.pkl")

In [19]:
dataset = dataset[dataset.index > '2010-01-01']
dataset.to_pickle("data/all_tickers_last_decade_features.pkl")

TypeError: '>' not supported between instances of 'tuple' and 'numpy.str_'